# Predict EE (Enantionmeric Excesses) in an end-to-end manner

*Zhongying Ru  zilla_ru@zju.edu.cn  Apr.27th*

### Input
SM1, SM2 (SMILES: Simplified Molecular Input Line Entry System)

### Output
predicted EE

### Loss Fuction
Measure the sum of difference from the predicted EE to the true EE of all training pairs (SM1, SM2)

### Train/Valid/Test set division
randomly divide as 8:1:1 (approximately)


In [ ]:
# prepare packages

from functools import partial
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import dgl
from dgl import model_zoo
from dgl.data.chem import BaseAtomFeaturizer
from dgl.data.chem import BaseBondFeaturizer
from dgl.data.chem import ConcatFeaturizer
from dgl.data.chem import one_hot_encoding
from dgl.data.chem.utils import atom_degree_one_hot
from dgl.data.chem.utils import atom_formal_charge
from dgl.data.chem.utils import atom_hybridization_one_hot
from dgl.data.chem.utils import atom_num_radical_electrons
from dgl.data.chem.utils import atom_total_num_H_one_hot
from dgl.data.chem.utils import atom_type_one_hot
from dgl.data.chem.utils import mol_to_bigraph
from rdkit import Chem
from torch.utils.data import DataLoader

## Load Train set

In [1]:
# input: SMILES strings of molecules
# intermediate output: molecule representation
import dgllife.model.gnn.attentivefp as AFP
AFP


Using backend: pytorch


## Define an AttentiveFP model

## Train and save the model

## Visualize the training process

## Load Test set

## Test and visualize